<h1> Wildebeest Detection using U-Net from VHR satellite images</h1>
Code Author: Zijing Wu 

***The code is developed for research project purposes.***

In [ ]:
#check the GPU colab assigns to you
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)


##  Load libraries

In [ ]:
import os
import rasterio


import rasterio.mask

import geopandas as gpd


import numpy as np
import matplotlib.pyplot as plt

import rioxarray as riox
# !pip install ipython-autotime
# %load_ext autotime

In [ ]:
#set the sys path where the modules locates
import sys
sys.path.insert(0,"core")

#If you are using Google Colaboratory, modify the path here
#sys.path.insert(0,"/content/drive/MyDrive/Colab/zijingwu-Satellite-based-monitoring-of-wildebeest/core")
from preprocess import *

from model import *

from evaluation import *

from visualization import *


from predict import *

## Define functions

In [ ]:
def generate_mask_from_point(raster_path, shape_path, output_path, file_name, cluster_size,plot):
    
    """Function that generates a binary mask from a point vector file (shp or geojson)
    
    raster_path = path to the .tif;

    shape_path = path to the shapefile or GeoJson of the point annotations.

    output_path = Path to save the binary mask.

    file_name = Name of the file.
    
    """
    
    #load raster
    if os.path.exists(output_path) == False:
        os.mkdir(output_path)
    with rasterio.open(raster_path, "r") as src:
        raster_img = src.read()
        bou = src.bounds
    
    #load o shapefile ou GeoJson
    true_pts = gpd.read_file(shape_path)
    if true_pts.crs == src.crs:
        print('The crs of the points are the same as the crs from the raster image!')   
        true_pts = gpd.read_file(shape_path, bbox = bou, encoding='utf-8')

    else:    
        print('The crs of the points are different from the crs from the raster image!')
        true_pts = gpd.read_file(shape_path, encoding='utf-8').to_crs(src.crs).clip_by_rect(*bou)    
                              
    print(len(true_pts.index))
    
    im_size = (src.meta['height'], src.meta['width'])
    mask = np.zeros(im_size)
    
    if len(true_pts.index) > 0:
    
        true_pts_x, true_pts_y = CrsToPixel(true_pts.geometry, src)
    
        for point in list(zip(true_pts_x,true_pts_y)):
    
            # print(point)
            x = point[0]
            y = point[1]
            if cluster_size ==16:
                mask[x-1:x+3,y-1:y+3] = 1
            elif cluster_size ==9:
                mask[x-1:x+2,y-1:y+2] = 1
            elif cluster_size ==25:
                mask[x-2:x+3,y-2:y+3] = 1
        
            #print(np.shape(mask))
    
    mask = mask.astype("uint8")
    
    bin_mask_meta = src.meta.copy()
    bin_mask_meta.update({'count': 1})
    os.chdir(output_path)
    with rasterio.open(file_name, 'w', **bin_mask_meta) as dst:
        dst.write(mask, 1)
    if plot == True:
        fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(8, 4),
                                                 sharex=True, sharey=True)
        ax1.imshow(np.moveaxis(raster_img, 0, -1))
        ax1.set_title('image')
        ax2.imshow(mask)
        ax2.set_title('mask')
        plt.tight_layout()
        plt.show()  

In [ ]:
#wildebeest -> prepare code for sampling: 
#roi: shp
#img: directory to the tiles
#iterate over rois: find overlapping tiles, if only one tile, clip it and get the patch; if two, get the strip id, check how roi intersects with strip boundary, if only intersects with one, clip the roi and the strip; if more than one, get the intersect rois, clip them separately and then merge it. Name it with the id
#iterate over rois: find annotation points within the roi, use roi to create the mask: point to pixel, expand the pixel (change values around the pixel）

# import rioxarray and shapley

def extract_image_from_AOI(image_path, roi_path, output_folder):

    # bound name: Bound2023_1.shp
    crs = "EPSG:32736"
    
    if os.path.exists(output_folder) == False:
        os.mkdirs(output_folder)
  
    roi = gpd.read_file(roi_path)   
    if roi.crs is None:
        roi = roi.set_crs(crs)

    roi = roi[~roi.is_empty] # remove potential empty polygons
    print("found {} valid geometries".format(len(roi)))

    file_name = os.path.split(image_path)[1]
    img_name, file_extension = os.path.splitext(file_name)

    file_name = os.path.split(roi_path)[1]
    roi_name, file_extension = os.path.splitext(file_name)
    

    for index, row in roi.iterrows():
        geom = row["geometry"]
        geom_bounds = geom.bounds
#     zone = box(*zone_bounds)
     
        out_image = os.path.join(output_folder, "{}_{}_{:0>6d}.tif".format(img_name, roi_name, index))
        if os.path.exists(out_image):
            print("Data already exists!")
            continue
                                 
#         print(*geom_bounds)
        test = rasterio.open(image_path)
#         print(test.crs)
#         print(test.bounds)
#         print(*rasterio.warp.transform_bounds(roi.crs, test.crs, *geom_bounds))        
        ras = riox.open_rasterio(image_path)
        clipped_raster = ras.rio.clip_box(*rasterio.warp.transform_bounds(roi.crs, test.crs, *geom_bounds))
        clipped_raster.rio.to_raster(out_image)  
        print("Saved image "+out_image)


## Define the data path

In [ ]:
#Change the path to your directory
Data_folder = "/home/zijing/wildebeest/SampleData/data_preparation"

#For Google Colaboratory users, update the directory:
#Data_folder = "/content/drive/MyDrive/Colab/Wildebeest-UNet/SampleData/1_Data_preparation/"

IMAGE_PATH = os.path.join(Data_folder, "images") #create folder to store the image patches
ROI_PATH = os.path.join(Data_folder, "roi/zone.shp") #save the ROI file here
POINT_PATH = os.path.join(Data_folder, "point/points.shp") #save the point annotations here
MASK_PATH = os.path.join(Data_folder, "masks") #create folder to store the mask patches

sate_image_path = "/home/zijing/wildebeest/data/202308/mosaic_image/2023_RGB.tif"

if not os.path.exists(IMAGE_PATH):
    os.makedirs(IMAGE_PATH)
if not os.path.exists(MASK_PATH):
    os.makedirs(MASK_PATH)


# Processing

## Extract images from the sampling plots

In [ ]:
extract_image_from_AOI(sate_image_path, ROI_PATH, IMAGE_PATH)

## Create the mask images from the source image and annotation AOIs

In [ ]:

for f in sorted(os.listdir(IMAGE_PATH)):
    #print(f)
    fdir = os.path.join(IMAGE_PATH, f)
    image_name, ext = os.path.splitext(f)
    if ext.lower() == ".tif":
        ID = image_name
        print(ID)
        generate_mask_from_point(fdir, POINT_PATH, MASK_PATH, ID+'.tif', 25,False)
        print("Generated mask image " + ID)
         


# References


***References:***

https://lpsmlgeo.github.io/2019-09-22-binary_mask/